In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


zip_path = 'C:/Users/starf/OneDrive/Bureau/FER-2013.zip' 
extract_path = 'dataset_extracted'  


if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted the dataset to {extract_path}")
else:
    print(f"Dataset already extracted at {extract_path}")


train_dir = os.path.join(extract_path, 'train')
test_dir = os.path.join(extract_path, 'test')


img_size = (48, 48)
batch_size = 64

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)


val_size = 0.2
val_batches = int(len(test_dataset) * val_size)
val_dataset = test_dataset.take(val_batches)
test_dataset = test_dataset.skip(val_batches)


def normalize_images(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(normalize_images)
val_dataset = val_dataset.map(normalize_images)
test_dataset = test_dataset.map(normalize_images)


train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  
])


model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    verbose=1
)


test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


model.save('emotion_detection_model.h5')


class_names = train_dataset.class_names
for images, labels in test_dataset.take(1):  
    predictions = model.predict(images)
    for i in range(5):  
        true_label = class_names[labels[i]]
        predicted_label = class_names[tf.argmax(predictions[i])]
        print(f"True Emotion: {true_label}, Predicted Emotion: {predicted_label}")
